# WSDM - KKBox's Churn Prediction Challenge
*Can you predict when subscribers will churn?*

* [Kaggle Page](https://www.kaggle.com/c/kkbox-churn-prediction-challenge/data)
* [Solution Sharing](https://www.kaggle.com/c/kkbox-churn-prediction-challenge/discussion/46078)

**Outline**

* [Read Data](#read)
* [Exploratory Data Analysis](#eda)
* [Feature Creation and Preprocessing](#preprocess)
* [Model and Score](#model) 
* [Predicition](#predict)
* [Reference](#reference)

---

In [2]:
%load_ext watermark

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

In [4]:
%watermark -a 'PredictiveII' -d -t -v -p pandas,numpy,sklearn,watermark

PredictiveII 2018-02-01 10:16:44 

CPython 3.6.3
IPython 6.1.0

pandas 0.20.3
numpy 1.13.3
sklearn 0.19.1
watermark 1.6.0


## <a id="read">Read Data</a>

In [5]:
def data_reader():
    """
    read data into notebook 
    """
        
    data_dir = os.path.join('..', 'data')

    train_path = os.path.join(data_dir, 'train_v2.csv')
    transactions_path = os.path.join(data_dir, 'transactions_v2.csv') 
    user_logs_path = os.path.join(data_dir, 'user_logs_v2.csv') 
    members_path = os.path.join(data_dir, 'members_v3.csv') 
    sample_submission_path = os.path.join(data_dir, 'sample_submission_v2.csv')
    user_label_201702_path = os.path.join(data_dir, 'kkbox-churn-scala-label/user_label_201702.csv')
    user_label_201703_path = os.path.join(data_dir, 'kkbox-churn-scala-label/user_label_201703.csv')

    train = pd.read_csv(train_path)
    transaction = pd.read_csv(transactions_path)
    user_log = pd.read_csv(user_logs_path)
    member = pd.read_csv(members_path)
    sample_submission = pd.read_csv(sample_submission_path)
    train_201702 = pd.read_csv(user_label_201702_path)
    train_201703 = pd.read_csv(user_label_201703_path)
    
    return train, transaction, user_log, member, sample_submission, train_201702, train_201703

In [6]:
def data_manipulate_transaction(transaction):
    """
    some transformation about the transaction data
    
    Parameters
    ----------
    transaction: pandas dataframe
    """
        
    # date retlated columns
    
    transaction['transaction_date'] = pd.to_datetime(transaction['transaction_date'], format='%Y%m%d', errors='ignore')
    #transaction['transaction_year'] = transaction['transaction_date'].apply(lambda x: x.year)
    #transaction['transaction_month'] = transaction['transaction_date'].apply(lambda x: x.month)
    #transaction['transaction_day'] = transaction['transaction_date'].apply(lambda x: x.day)
    transaction['membership_expire_date'] = pd.to_datetime(transaction['membership_expire_date'], format='%Y%m%d', errors='ignore')
    #transaction['membership_expire_year'] = transaction['membership_expire_date'].apply(lambda x: x.year)
    #transaction['membership_expire_month'] = transaction['membership_expire_date'].apply(lambda x: x.month)
    #transaction['membership_expire_day'] = transaction['membership_expire_date'].apply(lambda x: x.day)
    
    return transaction

In [7]:
train, transaction, user_log, member, sample_submission, train_201702, train_201703 = data_reader()
transaction = data_manipulate_transaction(transaction)

> **Merging the original train to transaction data**

There are **37382, 3.8%** of users from the train data can not be matched to their transaction records.

For those who can be matched to their transaction record, only **6.2%** of the user is_churn

In [10]:
train_transaction = pd.merge(train, transaction, on='msno', how='left')

In [11]:
len(train_transaction.msno.unique())

970960

In [12]:
train_transaction = train_transaction[train_transaction['payment_method_id'].notnull()]

In [13]:
len(train_transaction.msno.unique())

933578

In [14]:
train_transaction.groupby(['is_churn']).agg({'msno': 'nunique'}).reset_index()

,is_churn,msno
0,0,875683
1,1,57895


In [15]:
57895/(875683+57895)

0.062014100589345506

> **Merging the train data downloaded [here](https://www.kaggle.com/infinitewing/kkbox-churn-scala-label/data). This should be a better one.**

There are **47810, 5.4%** of users from the train data can not be matched to their transaction records.

For those who can be matched to their transaction record, only **0.3%** of the user is_churn

In [16]:
#train_transaction_02 = pd.merge(train_201702, transaction, on='msno', how='left')
train_transaction_03 = pd.merge(train_201703, transaction, on='msno', how='left')

In [17]:
#len(train_transaction_02.msno.unique())
len(train_transaction_03.msno.unique())

886500

In [18]:
#train_transaction_02 = train_transaction_02[train_transaction_02['payment_method_id'].notnull()]
train_transaction_03 = train_transaction_03[train_transaction_03['payment_method_id'].notnull()]

In [19]:
len(train_transaction_03.msno.unique())

853152

In [21]:
train_transaction_03.groupby(['is_churn']).agg({'msno': 'nunique'}).reset_index()

,is_churn,msno
0,False,842835
1,True,10317


In [51]:
10317/(842835+10317)

0.012092804095870373

## **Train**

**Description**

The train data consists of users whose subscription expires within the month of February 2017
In other words, these are the user ids whose subscription expires in February 2017. When we merge them with their transaction records, their lastest expire date should be the month of February 2017. We should have some more data related to their previous behavior after merging with transaction datatset.

**Some Finding**

* Originally, around 9% of the user in the training data is_churn

In [45]:
train.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [10]:
train.shape

(970960, 2)

In [37]:
train.is_churn.value_counts()

0    883630
1     87330
Name: is_churn, dtype: int64

In [38]:
train.is_churn.value_counts()[1]/train.shape[0]

0.089941913158111564

## train_201702

In [22]:
#train_201702.is_churn.value_counts()
train_201703.is_churn.value_counts()

False    842835
True      43665
Name: is_churn, dtype: int64

In [23]:
#train_201702.is_churn.value_counts()[1]/train_201702.shape[0]
train_201703.is_churn.value_counts()[1]/train_201703.shape[0]

0.049255499153976312

In [24]:
#train_check = pd.merge(train, train_201702, on='msno', how='outer').rename(columns={'is_churn_x':'is_churn_old','is_churn_y':'is_churn_new'})
train_check = pd.merge(train, train_201703, on='msno', how='outer').rename(columns={'is_churn_x':'is_churn_old','is_churn_y':'is_churn_new'})


In [212]:
train_check.head()

,msno,is_churn_old,is_churn_new
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1.0,NaN
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1.0,NaN
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1.0,NaN
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,NaN
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1.0,NaN


In [26]:
train_check.fillna(3.0, inplace=True)
train_check.loc[train_check['is_churn_new'] ==True, 'is_churn_new'] = 1.0
train_check.loc[train_check['is_churn_new'] ==False, 'is_churn_new'] = 0.0

In [27]:
train_check.head()

,msno,is_churn_old,is_churn_new
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1.0,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1.0,0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1.0,0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,3
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1.0,3


In [28]:
pd.crosstab(train_check.is_churn_old, train_check.is_churn_new)

is_churn_new,0.0,1.0,3.0
is_churn_old,,,
0.0,791104,8438,84088
1.0,28373,34243,24714
3.0,23358,984,0


3.0 represent NA.
There are 36156 user being labeled as churn but is labeled as not churn in the new version. The user id in thw two version are a lot different. There are 59424 users in new version and did not included in the old version.

---

## Transaction

Refreshed 11/06/2017, contains the transactions data until 3/31/2017.

* **msno**: user id
* **payment_method_id**: payment method
* **payment_plan_days**: length of membership plan in days
* **plan_list_price**: in New Taiwan Dollar (NTD)
* **actual_amount_paid**: in New Taiwan Dollar (NTD)
* **is_auto_renew**
* **transaction_date**: format %Y%m%d. (should be the date this user make a payment)
* **membership_expire_date**: format %Y%m%d. (the date that the membership expires before the payment?)
* **is_cancel**: whether or not the user canceled the membership in this transaction.



In [41]:
transaction.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0


In [42]:
transaction.shape

(1431009, 9)

> **Check who are the member that make lots of transaction and check their transaction history**



We see that there are many users is labeled as churn but the memebership expires after February 2017. One possible reason that can make this happen is that their previous membership expires in February 2017 and they did renew their subscription within 30 days. 

In [57]:
train_transaction.groupby(['msno']).agg({'payment_method_id':'count'}).rename(columns={'payment_method_id':'count'}).sort_values(by=['count'],ascending=False).head(10)

,count
msno,
72gJqt1O31E/WoxAEYFn9LHNI6mAZFGera5Q6gvsFkA=,208
5ty4nZkq54z93wQtBN7RHVYj8rNghBDCVBH+3xmxf0I=,172
OGKDrZQDB3yewZhoSd5qqvmG5A1GcNTYMexO95NlH+g=,148
WHsCtkOVsauvqBL0ULuG38887y7aU8GXdCmJMjw6hjQ=,145
SNlFRAsmUqnXKPofSXA8WYUc5DtmLcUMy4pXSJ3Ohz0=,131
vf6eQrnFfiS9o1kB/gRUJ4iTUixS9tHNKizqQ/1vLDI=,117
hQirtFbRr8t5T+/aVbrXM2tRNvoOIzChx5qzm1v/PE0=,104
gzTushiuaWZwJShNepFWD+O9a7C76pAfW7Vc07jHXw8=,89
IO+048XfFoL7X++ftIyb/zFKwNUUAkKaZLWrvg48TXA=,89


In [47]:
train_transaction.query('msno=="8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
13,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-08-08,2017-04-08,0.0
11,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-08-08,2017-05-09,0.0
4,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-09-08,2017-06-08,0.0
5,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-09-08,2017-07-08,0.0
9,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-10-08,2017-08-08,0.0
8,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-10-08,2017-09-08,0.0
7,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-11-08,2017-10-08,0.0
10,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-11-08,2017-11-07,0.0
6,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-12-08,2017-12-08,0.0
12,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-12-08,2018-01-08,0.0


In [95]:
train_transaction.query('msno=="u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)


,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
65705,u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY=,1,40.0,30.0,149.0,149.0,1.0,2017-03-01,2017-02-27,1.0
65704,u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY=,1,36.0,30.0,180.0,180.0,1.0,2017-03-01,2017-03-31,0.0


> **Who are the users that are churn and renew their service after 30 days of their previous membership expires? How many?**

In [78]:
train_churn = train_transaction.query('is_churn==1')

In [79]:
len(train_churn.msno.unique())

57895

The following table shows the minimum expire date for user who is labeled as churn

In [122]:
train_churn.groupby(['msno']).agg({'membership_expire_date':'min', 'payment_method_id':'count'}).rename(columns={'payment_method_id':'count'}).reset_index().sort_values(by=['membership_expire_date']).head()

# .query('membership_expire_date<"2017-03-31"')

,msno,membership_expire_date,count
52599,u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY=,2017-02-27,2
30034,V5A7FOae6HDZxBkVff+vJCKl52m6ZCaX43rgN4FjE1k=,2017-02-27,3
49588,qjAJoMTxL+6XV+DzXtG8ajXsDgxxySPqkbqeazn8bUM=,2017-02-28,1
28301,TBmKu6m5INtevSEZj7pCqI7iNBtNiNdy5AQpcizoEPo=,2017-02-28,1
27529,SHEIN+1JgjfdxVQdiSea8kGIE0moBTGvtoLdXBPetBA=,2017-02-28,7


> **Check the same thing using the new train data, train_201702**

In [29]:
#train_churn_02 = train_transaction_02.query('is_churn==1')
#len(train_churn_02.msno.unique())

train_churn_03 = train_transaction_03.query('is_churn==1')
len(train_churn_03.msno.unique())

10317

The following table shows the minimum expire date for user who is labeled as churn

In [55]:
#train_churn_02.groupby(['msno']).agg({'membership_expire_date':'min', 'payment_method_id':'count'}).rename(columns={'payment_method_id':'count'}).reset_index().sort_values(by=['membership_expire_date']).query('count==1').head()


train_churn_03.groupby(['msno']).agg({'membership_expire_date':'min', 'payment_method_id':'count'}).rename(columns={'payment_method_id':'count'}).reset_index().sort_values(by=['membership_expire_date']).query('count==1').shape
#.query('membership_expire_date>"2017-03-31"').shape


(10230, 3)

In [130]:
train_transaction_02.query('msno=="u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)


,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
50601,u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY=,False,40.0,30.0,149.0,149.0,1.0,2017-03-01,2017-02-27,1.0
50600,u7sL3DvERt0caPg2dKGHiD8NfNdpZLm2FPNdgwFreBY=,False,36.0,30.0,180.0,180.0,1.0,2017-03-01,2017-03-31,0.0


In [334]:
train_transaction_02.query('msno=="AbG12J8+duO7mxS/6jlV2CKcabvLBUEUkvMNmfBx08A="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
926160,AbG12J8+duO7mxS/6jlV2CKcabvLBUEUkvMNmfBx08A=,True,40.0,30.0,149.0,149.0,1.0,2017-03-01,2017-02-27,1.0
926159,AbG12J8+duO7mxS/6jlV2CKcabvLBUEUkvMNmfBx08A=,True,36.0,30.0,180.0,180.0,1.0,2017-03-31,2017-04-30,0.0


In [236]:
train_churn_02.membership_expire_date.describe()

count                    3412
unique                    502
top       2017-04-30 00:00:00
freq                      185
first     2017-02-27 00:00:00
last      2022-03-05 00:00:00
Name: membership_expire_date, dtype: object

In [237]:
train_transaction_02.membership_expire_date.describe()

count                  911399
unique                   1696
top       2017-04-30 00:00:00
freq                   108498
first     2016-04-24 00:00:00
last      2025-10-02 00:00:00
Name: membership_expire_date, dtype: object

In [48]:
#train_transaction_02.query('msno=="Ffb4sEnYuz00Uyc1NRP6/IpROMkZpEwIPGc/xsAcmxw="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)

train_transaction_03.query('msno=="P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)

#train_transaction_03.query('msno=="FCAKQEhmsJ423mUbEZ/YaHnBTP2sAU2FEiz83Z4omx8="').sort_values(by=['transaction_date','membership_expire_date'], ascending=True)


,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
945601,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,True,28.0,100.0,500.0,500.0,0.0,2017-02-23,2017-07-03,0.0


**Some finding**: 

Using the new labeled data, there are only 2829 users(0.3%) labeled as churn. We see that there are many users is labeled as churn but the memebership expires after February 2017. One possible reason that can make this happen is that their previous membership expires in February 2017 and they did renew their subscription within 30 days.

Also, it seems that we don't have the transaction data for the transaction dates of which the membership_expire_date is before Feb 2017. In other words, we do not have these users behavoir before Feb 2017.

---

## user_log

daily user logs describing listening behaviors of a user. Data collected until 2/28/2017.

* **msno**: user id
* **date**: format %Y%m%d
* **num_25**: # of songs played less than 25% of the song length
* **num_50**: # of songs played between 25% to 50% of the song length
* **num_75**: # of songs played between 50% to 75% of of the song length
* **num_985**: # of songs played between 75% to 98.5% of the song length
* **num_100**: # of songs played over 98.5% of the song length
* **num_unq**: # of unique songs played
* **total_secs**: total seconds played

**Some Finding**:

* There are 234707 out of 879537 users from the train_02 can not be matched to the user_log table.

In [38]:
user_log['date'] = pd.to_datetime(user_log['date'], format='%Y%m%d', errors='ignore')

In [49]:
user_log.query('msno=="P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY="').sort_values(by=['date'])

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
9707774,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-01,0,0,0,0,3,3,675.730
13669181,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-03,4,0,1,0,13,12,3413.379
13168499,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-04,1,1,1,0,25,27,6399.763
16351222,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-06,1,0,0,0,11,12,2605.405
3891278,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-07,1,2,4,0,17,15,4710.009
7923366,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-10,2,0,0,1,14,16,3635.931
4982320,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-11,0,0,0,0,60,60,14396.217
4825521,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-14,0,1,0,0,10,11,2292.703
12741332,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-16,1,1,1,2,12,15,3389.358
17157973,P6ZmR8GabfFJ2D2uWaPDw8brfRXz81GfpCvQc10pzZY=,2017-03-17,0,1,2,0,43,37,10415.510


In [257]:
user_log.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558


In [ ]:
user_log_group = user_log.groupby(['msno']).agg({'num_25':np.mean, 
                                'num_50':np.mean, 
                                'num_75':np.mean, 
                                'num_985':np.mean, 
                                'num_100':np.mean, 
                               'num_unq': np.mean,
                               'total_secs': np.mean}).reset_index()

In [259]:
train_transaction_02.msno.isin(user_log_group.msno).value_counts()

True     701787
False    209612
Name: msno, dtype: int64

In [260]:
train_user_02 = pd.merge(train_201702, user_log_group, on='msno', how='left')

In [261]:
train_user_02.shape

(879537, 9)

In [316]:
train_user_02.head()

,msno,is_churn,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,False,14.000000,1.000000,0.000000,0.000000,9.000000,24.000000,2362.332000
2,+/namlXq+u3izRjHCFJV4MgqcXcLidZYszVsROOq/y4=,False,42.173913,2.521739,1.086957,0.869565,30.217391,72.304348,8540.991739
3,+0/X9tkmyHyet9X80G6GTrDFHnJqvai8d1ZPhayT0os=,False,11.500000,0.833333,0.633333,1.033333,34.966667,24.566667,8823.093700
4,+09YGn842g6h2EZUXe0VWeC4bBoCbDGfUboitc0vIHw=,False,4.166667,1.125000,0.500000,0.583333,48.458333,42.541667,12348.791125
5,+0l+FDuhyjaZnu0APnrg5L9QqgaRw4RmdQMvqOtKDmU=,False,10.857143,1.761905,1.000000,0.904762,31.380952,36.142857,8320.127333


In [313]:
train_user_02 = train_user_02[train_user_02['num_25'].notnull()]

In [314]:
train_user_02.shape

(644830, 9)

In [315]:
train_user_02.groupby(['is_churn']).agg({'msno': 'nunique'})

,msno
is_churn,
False,639008
True,5822


---

## member

user information. Note that not every user in the dataset is available. Refreshed 11/13/2017, replaces members.csv data with the expiration date data removed.

* **msno**
* **city**
* **bd**: age. Note: this column has outlier values ranging from -7000 to 2015, please use your judgement.
* **gender**
* **registered_via**: registration method
* **registration_init_time**: format %Y%m%d
* **expiration_date**: format %Y%m%d, taken as a snapshot at which the member.csv is extracted. Not representing the actual churn behavior.


**Some Finding**

* There are 110860 out of 879537 users from the train_02 can not be matched to the member table.
* The value of gender for around 60% of user are NA.

In [280]:
member.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [273]:
member.shape

(6769473, 6)

In [274]:
len(member.msno.unique())

6769473

In [276]:
train_member_02 = pd.merge(train_201702, member, on='msno', how='left')

In [277]:
train_member_02.shape

(879537, 7)

In [278]:
train_member_02 = train_member_02[train_member_02['city'].notnull()]

In [279]:
train_member_02.shape

(768677, 7)

In [281]:
879537-768677

110860

In [282]:
pd.isnull(member).any(axis=0)

msno                      False
city                      False
bd                        False
gender                     True
registered_via            False
registration_init_time    False
dtype: bool

In [287]:
train_member_02.gender.isnull().value_counts()

True     457471
False    311206
Name: gender, dtype: int64

In [324]:
train_member_02.groupby(['is_churn']).agg({'msno':'nunique'})

,msno
is_churn,
False,736667
True,32010


In [325]:
32010/879537

0.036394148284836225

**Questions**

* According to the discussion board, it seems that we can generate the training data by our self using transaction data. However, in the transaction data, we can not really determine whether if a user is churn or not. 

    Quote from [here](https://www.kaggle.com/c/kkbox-churn-prediction-challenge/discussion/39756)


    The provided training data set is derived from transaction log. We picked the users who have their expiration dates fall in Feb, 2017 and check whether those people renew their subscription with 30 days after expiration to generate training label. Our method is not the only way to generate the training data. The training data set can be generate using different logic. Say, you can check each user's transaction log and calculate the interval between two consecutive entries. In this case, you will generate a training data set much bigger than what we provided in the data section.

* For many user who is labeled as churn, we do not have their transaction history before they churn. For example, some user can be labeled as churn and in the transaction data we only have some data with transaction date over Feb 2017. We shouldn't use the transaction data that is after Feb 2017 to predict whether if a user will churn or not in March. **Main problem: How can we use transaction data?**





In [308]:
train_transaction_02.query('msno=="+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc="')

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
78974,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,False,41.0,30.0,149.0,149.0,1.0,2017-03-26,2017-04-26,0.0


In [307]:
train_201702.query('msno=="+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc="')

,msno,is_churn
72183,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,False


In [335]:
transaction.sort_values(by=['msno', 'transaction_date', 'membership_expire_date']).head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
249714,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,22,395,1599,1599,0,2016-10-23,2018-02-06,0
1015786,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,41,30,99,99,1,2017-03-15,2017-04-15,0
1030112,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,1,2017-02-28,2017-04-19,0
335565,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,39,30,149,149,1,2017-03-31,2017-05-19,0
1173555,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,41,30,149,149,1,2017-03-26,2017-04-26,0


In [217]:
sample_merge = pd.merge(sample_submission,transaction, on='msno',how='left')

In [218]:
sample_merge.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,41.0,30.0,99.0,99.0,1.0,2017-03-18,2017-04-18,0.0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,34.0,30.0,149.0,149.0,1.0,2017-03-31,2017-04-30,0.0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,41.0,30.0,99.0,99.0,1.0,2017-03-15,2017-04-15,0.0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,41.0,30.0,99.0,99.0,1.0,2017-03-27,2017-04-27,0.0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,30.0,30.0,129.0,129.0,1.0,2017-03-22,2017-04-21,0.0


In [221]:
len(sample_submission.msno.unique())

907471

In [220]:
len(sample_merge[sample_merge['payment_method_id'].notnull()]['msno'].unique())

907470

In [54]:
sample_submission.is_churn.value_counts()

0    907471
Name: is_churn, dtype: int64

## <a id="eda">Exploratory Data Analysis</a>

* How to generate the submission? Where do those 907471 msno's coming from?
* The train data consists of users whose subscription expires within the month of February 2017. When I merge with transaction data, the menbership expire date for those ids have a lot of different membership_expire_date. What does that mean? Shouldn't all these id in the train data with the membership_expire_date in 2017 Feb?
* Don't understand what they are discuess [here](https://www.kaggle.com/c/kkbox-churn-prediction-challenge/discussion/39756)
* Should spend more time on reading the [Should I stay or should I go? - KKBox EDA
](https://www.kaggle.com/headsortails/should-i-stay-or-should-i-go-kkbox-eda), [Churn or No Churn - Exploration Data Analysis
](https://www.kaggle.com/rastaman/churn-or-no-churn-exploration-data-analysis)
* For some records in transaciton, the membership_expire_date is far from today, such as in the year of 2020, but they are labeled as churn. What does that mean? Remind that the criteria of "churn" is **no new valid service subscription within 30 days after the current membership expires.**
